In [ ]:
import scipy.io

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from kpnet.handler import *
from kpnet.network import *
from kpnet.signal import *
from kpnet.callback import *

In [ ]:
time_interval = 2 ** 16
chunk = 2 ** 13

In [ ]:
filelist  = pd.read_csv('data/hfd5/desc/data_gen.csv')

In [ ]:
filelist

In [ ]:
np.random.seed(0)
N = 64
array = np.zeros(shape=(time_interval, N, 1))
signal = SwitchSignal([2000, time_interval], [RandomUnitSignal(N), ZeroSignal(N)])
for i in range(time_interval):
    array[i] = signal[i]
d = {}
d['signal'] = array
scipy.io.savemat('data/mat/signal.mat', d)

In [ ]:
for i in tqdm(range(filelist.shape[0])):
    np.random.seed(0)
    N = filelist['N'][i]
    net = KPNetworkTanh(N, filelist['alpha'][i], filelist['beta'][i], filelist['gamma'][i])
    callbacks = {}
    callbacks['W'] = WeightsCallback(time_interval, N, chunk, 'data/hfd5/' + filelist['Filename'][i])
    signal = SwitchSignal([2000, time_interval], [RandomUnitSignal(N), ZeroSignal(N)])
    handler = ComputationHandler(time_interval)
    results = handler.run(net, signal, callbacks)